# Dogs Vs Cats

### Prerequisites
The assumption is that you have downloaded the data from the Kaggle Data Set. The following code used the data from Kaggle to create the classifier. In addition, it would be appropriate to visit the fastai course to learn more about its implementation.

In [6]:
# Import the necessary packages.
from fastai.vision import *
import zipfile

In [2]:
# Use the following code sequence to install the Kaggle API
#!pip install kaggle                                # Installs Kaggle API to download package and configure environment.
#! mkdir -p ~/.kaggle/
#! mv kaggle.json ~/.kaggle/
#!kaggle competitions download -c dogs-vs-cats      # Command that actually downloads files

In [4]:
# Unzip the data from Kaggle
def unzip(path):
    
    # Function Description: Given a path name to a folder, unzip the folder.
    # Function Parameters: path (The path to the folder)
    # Function Throws / Returns: Nothing
    
    with zipfile.ZipFile(path, 'r') as zip_ref:
        zip_ref.extractall('.')
        
#unzip('dogs-vs-cats.zip')         # Unpack the test and training data.
#unzip('train.zip')

In [ ]:
# Read the data from the training folder.
path_img = Path('train')             # Read the names of all the picture files.
fnames = get_image_files(path_img)
print(fnames[-2:])      # Dogs
print(fnames[:2])       # Cats

In [7]:
# Create a regex string that tells the function how to parse the data.
np.random.seed(2)
reg = r'/([^/]+)\.\d+.jpg$'

In [ ]:
# Label the data and normalise the images.
data = ImageDataBunch.from_name_re(path_img, fnames, pat=reg, ds_tfms=get_transforms(), size=224)
data.normalize(imagenet_stats)

In [ ]:
# Show the data and print the number classes. (Should be only two)
data.show_batch(rows=3, figsize=(5,7))
print(data.classes),data.c

In [ ]:
# Create a Convoluted Neural Network
learn = cnn_learner(data, models.resnet34, metrics=error_rate)

In [ ]:
# Train the Neural Net
learn.fit_one_cycle(4)
learn.save("first-net")

In [ ]:
interp = ClassificationInterpretation.from_learner(learn)

In [ ]:
interp.plot_top_losses(9, figsize=(11, 15))
# Plot the images that were classified incorrectly. It is evident that some of the pictures can use some enhancement (touch ups), 
# Or some sort of rescaleling. (Prediction, actual, loss, probability)

In [ ]:
learn.load("first-net")
learn.lr_find()    # Time to train the entire model.

In [ ]:
# Plot the learing curve.
learn.recorder.plot()

In [ ]:
# Train the entire model with adjusted learning rate.
learn.unfreeze()        
learn.fit_one_cycle(2, max_lr=slice(1e-7, 1e-5))

In [ ]:
# Setting up data for Resnet50 - Use a smaller bs if you run out of memory (bs)
data = ImageDataBunch.from_name_re(path_img, fnames, pat=reg, ds_tfms=get_transforms(), size=224)
data.normalize(imagenet_stats)
data.show_batch(rows=3, figsize=(5,7))
print(data.classes),data.c

In [8]:
# Attempt to use Resnet50
learn = cnn_learner(data, models.resnet50, metrics=error_rate)

NameError: name 'data' is not defined

In [ ]:
# Fit the model
learn.fit_one_cycle(5)

In [ ]:
# Save the model and print the learning function
learn.save("res50")
learn.lr_find()
learn.recorder.plot()

In [ ]:
# Unfreeze and fit once more.
learn.unfreeze()
learn.fit_one_cycle(3, max_lr=slice(1e-6,1e-3))